<a href="https://colab.research.google.com/github/franfgv9/Milestone_1_PLN/blob/main/Proyecto_2_PLN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --upgrade gensim

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.9/27.9 MB 48.4 MB/s eta 0:00:00


In [ ]:
!pip install emoji

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 608.4/608.4 kB 11.7 MB/s eta 0:00:00


In [ ]:
# =============================================================================
# MILESTONE 2 - HATEBR TEXT CLASSIFICATION (Simplified)
# Dataset: HateBR[](https://huggingface.co/datasets/franciellevargas/HateBR)
# Baselines:
#   A) TF-IDF (1-2 n-grams) + linguistic features + LogisticRegression
#   B) GloVe Average Embeddings + linguistic features + RandomForestClassifier
# =============================================================================

import pandas as pd
import numpy as np
import re
import spacy
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, f1_score
from sklearn.preprocessing import StandardScaler
from scipy.sparse import hstack, csr_matrix
from gensim.models import KeyedVectors
import emoji
import warnings
warnings.filterwarnings('ignore')

1. CARGA DEL DATASET (como en Proyecto 1)

In [ ]:
# ============================================================
# 1. CARGAR DATASET DESDE GITHUB (como en Proyecto 1)
# ============================================================

# URL "raw" del archivo HateBR.csv en GitHub
url = "https://raw.githubusercontent.com/franfgv9/Milestone_1_PLN/refs/heads/main/HateBR.csv"

# Leer el CSV directamente desde GitHub
df = pd.read_csv(url)

# Verificar la carga
print("Dimensiones del dataset:", df.shape)
print("Columnas disponibles:", df.columns.tolist())
df.head()

Dimensiones del dataset: (7000, 8)
Columnas disponibles: ['id', 'comentario', 'anotator1', 'anotator2', 'anotator3', 'label_final', 'links_post', 'account_post']


,id,comentario,anotator1,anotator2,anotator3,label_final,links_post,account_post
0,1,Mais um lixo,1,1,1,1,https://www.instagram.com/p/B2uThqdH9xI/,Carla Zambelli
1,2,Essa nao tem vergonha na cara!!,1,1,1,1,https://www.instagram.com/p/B2uThqdH9xI/,Carla Zambelli
2,3,Essa mulher é doente.pilantra!,1,1,1,1,https://www.instagram.com/p/B2uThqdH9xI/,Carla Zambelli
3,4,Comunista safada...,1,1,1,1,https://www.instagram.com/p/B2uThqdH9xI/,Carla Zambelli
4,5,Vagabunda. Comunista. Mentirosa. O povo chilen...,1,1,1,1,https://www.instagram.com/p/B2uThqdH9xI/,Carla Zambelli


In [ ]:
# ============================================================
# 2. SELECCIONAR COLUMNAS RELEVANTES
# ============================================================

df = df[["id", "comentario", "label_final"]]

# Verificar el resultado
print("\nColumnas seleccionadas:", df.columns.tolist())
print("Dimensiones del dataset:", df.shape)
df.head()


Columnas seleccionadas: ['id', 'comentario', 'label_final']
Dimensiones del dataset: (7000, 3)


,id,comentario,label_final
0,1,Mais um lixo,1
1,2,Essa nao tem vergonha na cara!!,1
2,3,Essa mulher é doente.pilantra!,1
3,4,Comunista safada...,1
4,5,Vagabunda. Comunista. Mentirosa. O povo chilen...,1


In [ ]:
# ============================================================
# 3. RENOMBRAR COLUMNAS PARA UNIFORMIDAD
# ============================================================

df = df.rename(columns={
    "comentario": "text",
    "label_final": "label"
})

print("\nColumnas finales:", df.columns.tolist())
df.head()

print("\nInformación del dataset:")
print(df.info())

# Limpiar valores nulos
df.dropna(subset=['text', 'label'], inplace=True)
df = df[df['label'].isin([0, 1])]  # 0: No ofensivo, 1: Ofensivo
df.reset_index(drop=True, inplace=True)

print(f"\nDataset final: {len(df)} ejemplos")
print("Distribución de clases:")
print(df['label'].value_counts())


Columnas finales: ['id', 'text', 'label']

Información del dataset:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7000 entries, 0 to 6999
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      7000 non-null   int64 
 1   text    7000 non-null   object
 2   label   7000 non-null   int64 
dtypes: int64(2), object(1)
memory usage: 164.2+ KB
None

Dataset final: 7000 ejemplos
Distribución de clases:
label
1    3500
0    3500
Name: count, dtype: int64


2. LIMPIEZA GENERAL (para features lingüísticas)

In [ ]:
# ============================================================
# 4. PREPROCESAMIENTO GENERAL (para features lingüísticas)
# ============================================================

def clean_for_features(text):
    text = re.sub(r'http[s]?://\S+', '', text)  # Quitar URLs
    text = re.sub(r'\s+', ' ', text).strip()    # Normalizar espacios
    return text

df['text_features'] = df['text'].apply(clean_for_features)

# Descargar modelo spaCy para portugués si no está instalado
!python -m spacy download pt_core_news_sm

# Cargar modelo spaCy para portugués
print("\nCargando modelo spaCy (pt_core_news_sm)...")
nlp = spacy.load("pt_core_news_sm")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 114.1 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('pt_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.

Cargando modelo spaCy (pt_core_news_sm)...


Por qué no más limpieza?
→ Porque queremos preservar mayúsculas, signos, emojis para extraer features.
Solo quitamos URLs y espacios extra → no afectan al análisis lingüístico.

3. EXTRACCIÓN DE FEATURES LINGÜÍSTICAS

In [ ]:
# ============================================================
# 5. EXTRACCIÓN DE FEATURES LINGÜÍSTICAS (con spaCy)
# ============================================================

def extract_linguistic_features(text):
    doc = nlp(text)
    features = {}

    # Básicas
    features['n_tokens'] = len(doc)
    features['upper_ratio'] = sum(1 for c in text if c.isupper()) / len(text) if text else 0
    features['n_exclam'] = text.count('!')
    features['n_question'] = text.count('?')

    # POS tags (solo adjetivos, sustantivos, verbos)
    pos_counts = {'ADJ': 0, 'NOUN': 0, 'VERB': 0}
    for token in doc:
        if token.pos_ in pos_counts:
            pos_counts[token.pos_] += 1
    total = sum(pos_counts.values())
    for pos in pos_counts:
        features[f'prop_{pos.lower()}'] = pos_counts[pos] / total if total > 0 else 0

    # Emojis
    features['n_emojis'] = len(emoji.emoji_list(text))

    return features

print("Extrayendo features lingüísticas...")
feat_list = df['text_features'].apply(extract_linguistic_features)
feat_df = pd.DataFrame(feat_list.tolist())
print(f"Features extraídas: {feat_df.shape[1]}")


Extrayendo features lingüísticas...
Features extraídas: 8


In [ ]:
print(feat_df.head())

   n_tokens  upper_ratio  n_exclam  n_question  prop_adj  prop_noun  \
0         3     0.083333         0           0      0.00       1.00   
1         8     0.032258         2           0      0.00       0.75   
2         5     0.033333         1           0      0.00       1.00   
3         3     0.052632         0           0      0.00       0.50   
4        15     0.051282         0           0      0.25       0.50   

   prop_verb  n_emojis  
0       0.00         0  
1       0.25         0  
2       0.00         0  
3       0.50         0  
4       0.25         0  


In [ ]:
# n_tokens    Comentarios largos → más contexto
# upper_ratio   MAYÚSCULAS = agresividad
# n_exclam, n_question    Exclamaciones = emoción fuerte
# prop_adj, prop_noun, prop_verb    Insultos suelen tener más adjetivos ("feio" "idiota")
# n_emojis       Emojis como risa o cara enojada refuerzan el tono

4. LIMPIEZA ESPECÍFICA POR MODELO

In [ ]:
# ============================================================
# 6. LIMPIEZA ESPECÍFICA POR MODELO
# ============================================================

# --- Modelo A: TF-IDF (superficial) ---
def clean_for_tfidf(text):
    text = re.sub(r'http[s]?://\S+', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

df['text_clean_tfidf'] = df['text'].apply(clean_for_tfidf)

# No se baja a minúsculas → lowercase=False en TfidfVectorizer
# Se preservan mayúsculas → "IDIOTA" ≠ "idiota" → señal de ofensa

# --- Modelo B: Embeddings (semántico) ---
emoji_map = {'[laughing face]': 'risada', '[angry face]': 'raiva', '[heart]': 'amor'}

def replace_emojis(text):
    for emj, word in emoji_map.items():
        text = text.replace(emj, f' {word} ')
    return text

def clean_for_embeddings(text):
    text = re.sub(r'http[s]?://\S+', '', text)
    text = replace_emojis(text)
    text = text.lower()
    text = re.sub(r'[^a-záéíóúâêôãõç\s]', ' ', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

# Baja a minúsculas → GloVe es case-sensitive
# Reemplaza emojis → "risada" en lugar de cara riendo
# Solo letras y acentos → compatibilidad con GloVe

df['text_clean_emb'] = df['text'].apply(clean_for_embeddings)

5. DIVISIÓN DE DATOS

In [ ]:
# ============================================================
# 7. PREPARAR DATOS PARA MODELOS
# ============================================================

X_text_tfidf = df['text_clean_tfidf']
X_text_emb = df['text_clean_emb']
X_features = feat_df.values
y = df['label'].values

# División train/test
X_text_tfidf_train, X_text_tfidf_test, \
X_text_emb_train, X_text_emb_test, \
X_feat_train, X_feat_test, \
y_train, y_test = train_test_split(
    X_text_tfidf, X_text_emb, X_features, y,
    test_size=0.2, random_state=42)

# Escalar features
scaler = StandardScaler()
X_feat_train_scaled = scaler.fit_transform(X_feat_train)
X_feat_test_scaled = scaler.transform(X_feat_test)

# Escalado: necesario para combinar con TF-IDF y embeddings

6. MODELO A: TF-IDF + FEATURES + LogisticRegression

In [ ]:
# ============================================================
# IMPORTS QUE FALTABAN (PON ESTO AL PRINCIPIO DEL NOTEBOOK, CON LOS DEMÁS)
# ============================================================
from sklearn.model_selection import GridSearchCV
from sklearn.svm import LinearSVC
# (Los demás ya los tenías: LogisticRegression, RandomForestClassifier, etc.)

# ============================================================
# 8. MODELO A: TF-IDF + FEATURES → 3 ALGORITMOS + GRIDSEARCH
# ============================================================

print("\n" + "="*70)
print("TRAINING MODEL A: TF-IDF (1-2 n-grams) + LINGUISTIC FEATURES")
print("3 CLASSIFIERS: LogisticRegression, RandomForest, LinearSVM")
print("WITH GridSearchCV FOR BEST HYPERPARAMETERS")
print("="*70)

# Vectorizador TF-IDF
tfidf = TfidfVectorizer(
    ngram_range=(1, 2),
    lowercase=False,
    max_features=10000
)

X_tfidf_train = tfidf.fit_transform(X_text_tfidf_train)
X_tfidf_test = tfidf.transform(X_text_tfidf_test)

# Combinar con features lingüísticas
X_train_A = hstack([X_tfidf_train, csr_matrix(X_feat_train_scaled)])
X_test_A = hstack([X_tfidf_test, csr_matrix(X_feat_test_scaled)])

print(f"TF-IDF vocabulary size: {len(tfidf.vocabulary_)} terms")
print(f"Final training matrix shape: {X_train_A.shape}")

# Resultados
results_A = {}

# 1. Logistic Regression
print("\nTraining Logistic Regression (GridSearch on C)...")
log_grid = GridSearchCV(
    LogisticRegression(max_iter=1000, class_weight='balanced', random_state=42),
    param_grid={'C': [0.1, 1.0, 10.0]},
    cv=3,
    scoring='f1_macro',
    n_jobs=-1
)
log_grid.fit(X_train_A, y_train)
y_pred_log = log_grid.predict(X_test_A)
f1_log = f1_score(y_test, y_pred_log, average='macro')
results_A['LogisticRegression'] = (f1_log, log_grid.best_params_)
print(f"   Best C: {log_grid.best_params_['C']} | F1-macro: {f1_log:.4f}")

# 2. Random Forest
print("Training Random Forest (GridSearch on n_estimators & max_depth)...")
rf_grid = GridSearchCV(
    RandomForestClassifier(random_state=42, n_jobs=-1),
    param_grid={
        'n_estimators': [100, 200, 300],
        'max_depth': [None, 20, 30]
    },
    cv=3,
    scoring='f1_macro',
    n_jobs=-1
)
rf_grid.fit(X_train_A, y_train)
y_pred_rf = rf_grid.predict(X_test_A)
f1_rf = f1_score(y_test, y_pred_rf, average='macro')
results_A['RandomForest'] = (f1_rf, rf_grid.best_params_)
print(f"   Best params: {rf_grid.best_params_} | F1-macro: {f1_rf:.4f}")

# 3. Linear SVM
print("Training Linear SVM (GridSearch on C)...")
svm_grid = GridSearchCV(
    LinearSVC(class_weight='balanced', max_iter=10000, random_state=42),
    param_grid={'C': [0.01, 0.1, 1.0, 10.0]},
    cv=3,
    scoring='f1_macro',
    n_jobs=-1
)
svm_grid.fit(X_train_A, y_train)
y_pred_svm = svm_grid.predict(X_test_A)
f1_svm = f1_score(y_test, y_pred_svm, average='macro')
results_A['SVM'] = (f1_svm, svm_grid.best_params_)
print(f"   Best C: {svm_grid.best_params_['C']} | F1-macro: {f1_svm:.4f}")

# RESUMEN
print("\n" + "="*60)
print("MODEL A SUMMARY (TF-IDF + Linguistic Features)")
print("="*60)
for name, (f1, params) in results_A.items():
    print(f"{name:18} → F1-macro: {f1:.4f} | Best params: {params}")
print("="*60)

best_A_name = max(results_A, key=lambda k: results_A[k][0])
best_A_f1 = results_A[best_A_name][0]
f1_A = best_A_f1 # Assign best_A_f1 to f1_A
print(f"WINNER MODEL A → {best_A_name} with F1-macro = {f1_A:.4f}")


TRAINING MODEL A: TF-IDF (1-2 n-grams) + LINGUISTIC FEATURES
3 CLASSIFIERS: LogisticRegression, RandomForest, LinearSVM
WITH GridSearchCV FOR BEST HYPERPARAMETERS
TF-IDF vocabulary size: 10000 terms
Final training matrix shape: (5600, 10008)

Training Logistic Regression (GridSearch on C)...
   Best C: 10.0 | F1-macro: 0.8114
Training Random Forest (GridSearch on n_estimators & max_depth)...
   Best params: {'max_depth': None, 'n_estimators': 100} | F1-macro: 0.7536
Training Linear SVM (GridSearch on C)...
   Best C: 1.0 | F1-macro: 0.8107

MODEL A SUMMARY (TF-IDF + Linguistic Features)
LogisticRegression → F1-macro: 0.8114 | Best params: {'C': 10.0}
RandomForest       → F1-macro: 0.7536 | Best params: {'max_depth': None, 'n_estimators': 100}
SVM                → F1-macro: 0.8107 | Best params: {'C': 1.0}
WINNER MODEL A → LogisticRegression with F1-macro = 0.8114


7. MODELO B: GloVe + FEATURES + RandomForest

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# ============================================================
# 9. MODELO B: GLOVE EMBEDDINGS + FEATURES (gensim)
# ============================================================

print("\n" + "="*60)
print("ENTRENANDO MODELO B: GLOVE + FEATURES")
print("="*60)

# Cargar GloVe (descargar desde NILC: http://nilc.icmc.usp.br/nwp/embeddings)
# Ejemplo: glove_s100.txt
glove_path = "/content/drive/MyDrive/proyecto2_PLN/glove_s100.txt"
print(f"Cargando GloVe desde: {glove_path}")
w2v_model = KeyedVectors.load_word2vec_format(glove_path)

def get_avg_embedding(text, model):
    words = text.split()
    valid = [w for w in words if w in model]
    if not valid:
        return np.zeros(model.vector_size)
    return np.mean([model[w] for w in valid], axis=0)

# Representación de frase: promedio de vectores
# Palabras fuera del vocabulario: ignoradas

print("Calculando embeddings promedio...")
X_emb_train = np.array([get_avg_embedding(t, w2v_model) for t in X_text_emb_train])
X_emb_test = np.array([get_avg_embedding(t, w2v_model) for t in X_text_emb_test])

X_train_B = np.hstack([X_emb_train, X_feat_train_scaled])
X_test_B = np.hstack([X_emb_test, X_feat_test_scaled])

print(f"GloVe embedding dimension: {X_emb_train.shape[1]}")
print(f"Final training matrix shape: {X_train_B.shape}")
# Resultados
results_B = {}

# 1. Logistic Regression
print("\nTraining Logistic Regression (GridSearch on C)...")
log_grid = GridSearchCV(
    LogisticRegression(max_iter=1000, class_weight='balanced', random_state=42),
    param_grid={'C': [0.1, 1.0, 10.0]},
    cv=3,
    scoring='f1_macro',
    n_jobs=-1
)
log_grid.fit(X_train_B, y_train)
y_pred_log = log_grid.predict(X_test_B)
f1_log = f1_score(y_test, y_pred_log, average='macro')
results_B['LogisticRegression'] = (f1_log, log_grid.best_params_)
print(f"   Best C: {log_grid.best_params_['C']} | F1-macro: {f1_log:.4f}")

# 2. Random Forest
print("Training Random Forest (GridSearch on n_estimators & max_depth)...")
rf_grid = GridSearchCV(
    RandomForestClassifier(random_state=42, n_jobs=-1),
    param_grid={
        'n_estimators': [100, 200, 300],
        'max_depth': [None, 20, 30]
    },
    cv=3,
    scoring='f1_macro',
    n_jobs=-1
)
rf_grid.fit(X_train_B, y_train)
y_pred_rf = rf_grid.predict(X_test_B)
f1_rf = f1_score(y_test, y_pred_rf, average='macro')
results_B['RandomForest'] = (f1_rf, rf_grid.best_params_)
print(f"   Best params: {rf_grid.best_params_} | F1-macro: {f1_rf:.4f}")

# 3. Linear SVM
print("Training Linear SVM (GridSearch on C)...")
svm_grid = GridSearchCV(
    LinearSVC(class_weight='balanced', max_iter=10000, random_state=42),
    param_grid={'C': [0.01, 0.1, 1.0, 10.0]},
    cv=3,
    scoring='f1_macro',
    n_jobs=-1
)
svm_grid.fit(X_train_B, y_train)
y_pred_svm = svm_grid.predict(X_test_B)
f1_svm = f1_score(y_test, y_pred_svm, average='macro')
results_B['SVM'] = (f1_svm, svm_grid.best_params_)
print(f"   Best C: {svm_grid.best_params_['C']} | F1-macro: {f1_svm:.4f}")

# RESUMEN FINAL MODELO B
print("\n" + "="*60)
print("MODEL B SUMMARY (GloVe avg + Linguistic Features)")
print("="*60)
for name, (f1, params) in results_B.items():
    print(f"{name:18} → F1-macro: {f1:.4f} | Best params: {params}")
print("="*60)

best_B_name = max(results_B, key=lambda k: results_B[k][0])
best_B_f1 = results_B[best_B_name][0]
f1_B = best_B_f1 # Assign best_B_f1 to f1_B
print(f"WINNER MODEL B → {best_B_name} with F1-macro = {f1_B:.4f}")
# F1-macro: promedio sin ponderar → justo con clases desbalanceadas
# digits=4: precisión para el informe


ENTRENANDO MODELO B: GLOVE + FEATURES
Cargando GloVe desde: /content/drive/MyDrive/proyecto2_PLN/glove_s100.txt
Calculando embeddings promedio...
GloVe embedding dimension: 100
Final training matrix shape: (5600, 108)

Training Logistic Regression (GridSearch on C)...
   Best C: 1.0 | F1-macro: 0.8090
Training Random Forest (GridSearch on n_estimators & max_depth)...
   Best params: {'max_depth': 20, 'n_estimators': 300} | F1-macro: 0.8071
Training Linear SVM (GridSearch on C)...
   Best C: 0.1 | F1-macro: 0.8097

MODEL B SUMMARY (GloVe avg + Linguistic Features)
LogisticRegression → F1-macro: 0.8090 | Best params: {'C': 1.0}
RandomForest       → F1-macro: 0.8071 | Best params: {'max_depth': 20, 'n_estimators': 300}
SVM                → F1-macro: 0.8097 | Best params: {'C': 0.1}
WINNER MODEL B → SVM with F1-macro = 0.8097


8. COMPARACIÓN TEÓRICA: ¿CUÁNDO GANA CADA MODELO?

In [ ]:
# ============================================================
# 10. COMPARACIÓN FINAL
# ============================================================
print("\n" + "="*70)
print("FINAL RESULTS - ALL 6 MODELS")
print("="*70)

# Recoger todos los F1
all_models = {}

# Modelo A (TF-IDF)
for name, (f1, params) in results_A.items():
    all_models[f"TF-IDF + {name}"] = f1

# Modelo B (GloVe)
for name, (f1, params) in results_B.items():
    all_models[f"GloVe + {name}"] = f1

# Ordenar de mejor a peor
ranking = sorted(all_models.items(), key=lambda x: x[1], reverse=True)

# Mostrar ranking limpio
print(f"{'Rank':<4} {'Model':<30} {'F1-macro'}")
print("-" * 50)
for i, (model, f1) in enumerate(ranking, 1):
    print(f"{i}.   {model:<30} {f1:.4f}")

# Ganador
best_model, best_f1 = ranking[0]
print("\n" + "="*70)
print(f"WINNER: {best_model}")
print(f"F1-macro = {best_f1:.4f}")
print("="*70)


FINAL RESULTS - ALL 6 MODELS
Rank Model                          F1-macro
--------------------------------------------------
1.   TF-IDF + LogisticRegression    0.8114
2.   TF-IDF + SVM                   0.8107
3.   GloVe + SVM                    0.8097
4.   GloVe + LogisticRegression     0.8090
5.   GloVe + RandomForest           0.8071
6.   TF-IDF + RandomForest          0.7536

WINNER: TF-IDF + LogisticRegression
F1-macro = 0.8114
